In [1]:
from collections import namedtuple
import numpy as np
import apollocaffe
from apollocaffe.layers import *

In [2]:
Hmm = namedtuple("Hmm", ["n_hidden", "n_observed", "transitions", "emissions"])

def make_hmm(n_hidden, n_observed):
    transitions = np.random.random((n_hidden, n_hidden))
    emissions = np.random.random((n_hidden, n_observed))
    
    for i in range(n_hidden):
        transitions[i, :] /= transitions[i, :].sum()
        emissions[i, :] /= emissions[i, :].sum()
        
    return Hmm(n_hidden, n_observed, transitions, emissions)

def sample(n_steps, hmm):
    current_state = np.random.choice(hmm.n_hidden)
    hidden = []
    observed = []
    for i in range(n_steps):
        hidden.append(current_state)
        observed.append(np.random.choice(hmm.n_observed, 
                                         p=hmm.emissions[current_state, :]))
        current_state = np.random.choice(hmm.n_hidden,
                                         p=hmm.transitions[current_state, :])
        
    return hidden, observed

In [3]:
def forward(observed, hmm, viterbi=False):
    chart = np.ones((hmm.n_hidden, len(observed)))
    for i in range(len(observed)):
        if i > 0:
            if viterbi:
                for j in range(hmm.n_hidden):
                    chart[j, i] *= (hmm.transitions[:, j] * chart[:, i-1]).max()
            else:
                chart[:, i] *= hmm.transitions.T.dot(chart[:, i-1])
        chart[:, i] *= hmm.emissions[:, observed[i]]
    return chart
        
def backward(observed, hmm, viterbi=False):
    chart = np.ones((hmm.n_hidden, len(observed)))
    for i in range(len(observed)-1, -1, -1):
        if i < len(observed) - 1:
            prev = chart[:, i+1] * hmm.emissions[:, observed[i+1]]
            if viterbi:
                for j in range(hmm.n_hidden):
                    chart[j, i] *= (hmm.transitions[j, :] * prev).max()
            else:
                chart[:, i] *= hmm.transitions.dot(prev)
    return chart
            
def forward_backward(observed, hmm, viterbi=False):
    chart_f = forward(observed, hmm, viterbi)
    chart_b = backward(observed, hmm, viterbi)
    marginals = chart_f * chart_b
    return marginals

In [4]:
def compute_chart_acc(hiddens, observeds, hmm, backward, viterbi):
    acc_f, exact_f = 0., 0.
    acc_fb, exact_fb = 0., 0.
    acc_f_v, exact_f_v = 0., 0.
    acc_fb_v, exact_fb_v = 0., 0.
    
    for i in range(hiddens.shape[0]):
        hidden = hiddens[i, :]
        observed = observeds[i, :]
        chart_f = forward(observed, hmm)
        chart_fb = forward_backward(observed, hmm)
        chart_f_v = forward(observed, hmm, viterbi=True)
        chart_fb_v = forward_backward(observed, hmm, viterbi=True)

        pred_f = chart_f.argmax(axis=0)
        pred_fb = chart_fb.argmax(axis=0)
        pred_f_v = chart_f_v.argmax(axis=0)
        pred_fb_v = chart_fb_v.argmax(axis=0)

        acc_f += (pred_f == hidden).sum()
        acc_fb += (pred_fb == hidden).sum()
        acc_f_v += (pred_f_v == hidden).sum()
        acc_fb_v += (pred_fb_v == hidden).sum()

        exact_f += (pred_f == hidden).all()
        exact_fb += (pred_fb == hidden).all()
        exact_f_v += (pred_f_v == hidden).all()
        exact_fb_v += (pred_fb_v == hidden).all()

        
        
    if viterbi:    
        if backward:
            return exact_fb_v / hiddens.shape[0]
        else:
            return exact_f_v / hiddens.shape[0]
    else:
        n = hiddens.shape[0] * hiddens.shape[1]
        if backward:
            return acc_fb / n
        else:
            return acc_f / n

In [5]:
class Index(PyLayer):
    def forward(self, bottom, top):
        data, indices = bottom
        index_data = indices.data.astype(int)
        top[0].reshape(indices.shape)
        top[0].data[...] = data.data[range(indices.shape[0]), index_data]

    def backward(self, top, bottom):
        data, indices = bottom
        index_data = indices.data.astype(int)
        data.diff[...] = 0
        data.diff[range(indices.shape[0]), index_data] = top[0].diff

In [18]:
N_LAYER = 100

net = apollocaffe.ApolloNet()

def iter_rnn(hiddens, observeds, hmm):
    net.clear_forward()
    
    seq_len = hiddens.shape[1]
    
    l_data = "data_%d"
    l_vec = "vec_%d"
    l_concat = "concat_%d"
    l_hidden = "hidden_%d"
    l_relu = "relu_%d"
    l_output = "output_%d"
    l_target = "target_%d"
    l_loss = "loss_%d"
    
    p_vec = ["p_vec_weight"]
    p_hidden = ["p_hidden_weight", "p_hidden_bias"]
    p_output = ["p_output_weight", "p_output_bias"]
    
    net.f(NumpyData("seed", np.zeros((hiddens.shape[0], N_LAYER))))
    l_prev = "seed"
    
    loss = 0
    acc = 0
    for i in range(seq_len):
        net.f(NumpyData(l_data % i, observeds[:, i]))
        net.f(NumpyData(l_target % i, hiddens[:, i]))
        net.f(Wordvec(l_vec % i, N_LAYER, hmm.n_observed,
                      bottoms=[l_data % i], param_names=p_vec))
        net.f(Concat(l_concat % i, bottoms=[l_vec % i, l_prev]))
        net.f(InnerProduct(l_hidden % i, N_LAYER, bottoms=[l_concat % i],
                           param_names=p_hidden))
        net.f(ReLU(l_relu % i, bottoms=[l_hidden % i]))
        l_prev = l_relu % i
        net.f(InnerProduct(l_output % i, hmm.n_hidden, 
                           bottoms=[l_relu % i], param_names=p_output))
        loss += net.f(SoftmaxWithLoss(l_loss % i, 
                                      bottoms=[l_output % i, l_target % i]))
        preds = net.blobs[l_output % i].data.argmax(axis=1)
        acc += np.mean(preds == hiddens[:, i])
        
    net.backward()
    net.update(lr=0.01)
    
    return loss / seq_len, acc / seq_len
        
def iter_bdrnn(hiddens, observeds, hmm, viterbi):
    net.clear_forward()
    
    seq_len = hiddens.shape[1]
    
    l_data = "data_%d_bd"
    l_vec = "vec_%d_bd"
    #
    l_concat_f = "concat_f_%d_bd"
    l_hidden_f = "hidden_f_%d_bd"
    l_relu_f = "relu_f_%d_bd"
    #
    l_concat_b = "concat_b_%d_bd"
    l_hidden_b = "hidden_b_%d_bd"
    l_relu_b = "relu_b_%d_bd"
    #
    l_concat_fb = "concat_fb_%d_bd"
    l_index = "index_%d_bd"
    l_output = "output_%d_bd"
    l_target = "target_%d_bd"
    l_softmax = "softmax_%d_bd"
    l_total_logprob = "total_logprob_bd"
    l_viterbi_target = "viterbi_target_bd"
    l_viterbi_loss = "total_loss_bd"
    l_loss = "loss_%d_bd"
    
    p_vec = ["p_vec_weight_bd"]
    p_hidden_f = ["p_hidden_f_weight_bd", "p_hidden_f_bias_bd"]
    p_hidden_b = ["p_hidden_b_weight_bd", "p_hidden_b_bias_bd"]
    p_output = ["p_output_weight_bd", "p_output_bias_bd"]
    
    net.f(NumpyData("seed_bd", np.zeros((hiddens.shape[0], N_LAYER))))
    
    l_prev = "seed_bd"
    for i in range(seq_len):
        net.f(NumpyData(l_data % i, observeds[:, i]))
        net.f(NumpyData(l_target % i, hiddens[:, i]))
        net.f(Wordvec(l_vec % i, N_LAYER, hmm.n_observed,
                      bottoms=[l_data % i], param_names=p_vec))
        
        net.f(Concat(l_concat_f % i, bottoms=[l_vec % i, l_prev]))
        net.f(InnerProduct(l_hidden_f % i, N_LAYER, bottoms=[l_concat_f % i],
                           param_names=p_hidden_f))
        net.f(ReLU(l_relu_f % i, bottoms=[l_hidden_f % i]))
        l_prev = l_relu_f % i
        
    l_prev = "seed_bd"
    for i in reversed(range(seq_len)):
        net.f(Concat(l_concat_b % i, bottoms=[l_vec % i, l_prev]))
        net.f(InnerProduct(l_hidden_b % i, N_LAYER, bottoms=[l_concat_b % i],
                           param_names=p_hidden_b))
        net.f(ReLU(l_relu_b % i, bottoms=[l_hidden_b % i]))
        l_prev = l_relu_b % i
        
    loss = 0
    acc = 0
    if viterbi:
        preds = []
        for i in range(seq_len):
            net.f(Concat(l_concat_fb % i, bottoms=[l_relu_f % i, l_relu_b % i]))
            net.f(InnerProduct(l_output % i, hmm.n_hidden, 
                               bottoms=[l_concat_fb % i], param_names=p_output))
            net.f(Softmax(l_softmax % i, bottoms=[l_output % i]))
            net.f(Index(l_index % i, {}, bottoms=[l_softmax % i, l_target % i]))
            preds.append(net.blobs[l_output % i].data.argmax(axis=1))
        # TODO all in logspace
        net.f(Eltwise(l_total_logprob, "PROD", bottoms=[l_index % i for i in range(seq_len)]))
        net.f(NumpyData(l_viterbi_target, np.ones(hiddens.shape[0])))
        loss += net.f(SigmoidCrossEntropyLoss(l_viterbi_loss, bottoms=[l_total_logprob, l_viterbi_target]))
        preds = np.asarray(preds).T
        acc += np.mean((preds == hiddens).min(axis=1))
            
    else:
        for i in range(seq_len):
            net.f(Concat(l_concat_fb % i, bottoms=[l_relu_f % i, l_relu_b % i]))
            net.f(InnerProduct(l_output % i, hmm.n_hidden, 
                               bottoms=[l_concat_fb % i], param_names=p_output))
            loss += net.f(SoftmaxWithLoss(l_loss % i, 
                                          bottoms=[l_output % i, l_target % i]))
            preds = net.blobs[l_output % i].data.argmax(axis=1)
            acc += np.mean(preds == hiddens[:, i])
        
    net.backward()
    net.update(lr=0.01)
    
    if viterbi:
        return loss, acc
    else:
        return loss / seq_len, acc / seq_len

def train_seq_rnn(hmm, backward, viterbi):
    loss = 0
    acc = 0
    
    c_acc = 0
    for i_iter in range(2000):
        pairs = [sample(5, hmm) for i in range(100)]
        hiddens, observeds = zip(*pairs)
        hiddens = np.asarray(hiddens)
        observeds = np.asarray(observeds)
        if backward:
            loss_i, acc_i = iter_bdrnn(hiddens, observeds, hmm, viterbi)
        else:
            assert viterbi == False
            loss_i, acc_i = iter_rnn(hiddens, observeds, hmm)
        loss += loss_i
        acc += acc_i
        
        c_acc += compute_chart_acc(hiddens, observeds, hmm, backward, viterbi)
        
        if i_iter % 100 == 0:
            print "%6.2f %6.2f | %6.2f" % (loss, acc, c_acc)
            loss, acc, c_acc = 0, 0, 0


In [21]:
np.random.seed(1)
apollocaffe.set_random_seed(1)
hmm = make_hmm(3, 3)
train_seq_rnn(hmm, backward=False, viterbi=False)

  1.07   0.54 |   0.61
 92.33  60.79 |  62.52
 86.96  62.11 |  62.56
 85.55  62.35 |  62.49
 85.21  62.79 |  62.79
 84.97  62.62 |  62.64
 84.93  62.69 |  62.70
 85.09  62.42 |  62.44
 85.22  62.57 |  62.60
 84.77  62.73 |  62.74
 85.13  62.59 |  62.57
 85.08  62.17 |  62.16
 85.12  62.27 |  62.31
 85.33  62.23 |  62.29
 84.81  62.69 |  62.80
 85.31  62.21 |  62.26
 85.67  62.09 |  62.14
 85.31  62.28 |  62.31
 84.36  62.75 |  62.77
 84.52  62.83 |  62.84


In [23]:
np.random.seed(1)
apollocaffe.set_random_seed(1)
hmm = make_hmm(3, 3)
train_seq_rnn(hmm, backward=True, viterbi=False)

  1.08   0.53 |   0.64
 93.33  60.23 |  63.24
 87.59  62.11 |  63.22
 85.50  62.10 |  63.01
 84.74  62.83 |  63.42
 84.30  62.98 |  63.14
 84.07  63.05 |  63.27
 84.10  62.96 |  63.10
 84.19  63.07 |  63.19
 83.68  63.14 |  63.30
 84.12  63.01 |  62.98
 84.10  62.73 |  62.69
 84.01  62.82 |  62.95
 84.28  62.68 |  62.86
 83.76  63.02 |  63.08
 84.28  62.59 |  62.65
 84.69  62.67 |  62.63
 84.31  62.68 |  62.63
 83.21  63.41 |  63.48
 83.41  63.29 |  63.33
